**Import Required Libraries**

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

**Scrape Wikipedia Data**

In [2]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
response = requests.get(url)

In [3]:
data = response.text
soup = BeautifulSoup(data,'html.parser')
wiki_table=soup.find('table')

In [4]:
df = pd.read_html(str(wiki_table))[0]
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


**Format and Clean the Data**

In [5]:
df.drop(0,inplace=True)

In [6]:
df.columns = ['PostalCode','Borough','Neighborhood']
df.head()

,PostalCode,Borough,Neighborhood
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"


In [7]:
df2=df[df['Borough'].str.contains("Not assigned") == False].reset_index()
df2.head()

,index,PostalCode,Borough,Neighborhood
0,2,M3A,North York,Parkwoods
1,3,M4A,North York,Victoria Village
2,4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,5,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


**Combine PostalCodes and Boroughs with the same values**

In [8]:
df3= df2.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(', '.join).reset_index()
df3.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


**Verify the number of rows and columns in the dataset**

In [9]:
df3.shape

(103, 3)

The dataset contains 103 rows and 3 columns

**Save the dataset**

In [10]:
df3.to_csv('Toronto_Part1.csv')

**Collect the geospacial data from https://cocl.us/Geospatial_data**

In [11]:
url='https://cocl.us/Geospatial_data/Geospatial_Coordinates.csv'
df_geo=pd.read_csv(url)
df_geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


**Format the column names as before**

In [12]:
df_geo = df_geo.rename(columns = {'Postal Code':'PostalCode'}) 
df_geo.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


**Upload the first dataset from Part 1**

In [13]:
df3=pd.read_csv('Toronto_Part1.csv', index_col=[0])
df3.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


**Merge both datasets**

In [14]:
df3 = pd.merge(df3, df_geo, on = 'PostalCode')
df3.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


**Verify the number of rows and columns in the new dataset**

In [15]:
df3.shape

(103, 5)

There are 103 rows and 5 columns

**Save the new dataset**

In [16]:
df3.to_csv('Toronto_Part2.csv', index=False)

**Upload the dataset from Part 2**

In [17]:
df3=pd.read_csv('Toronto_Part2.csv')
df3.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


**Sort the data to show only Boroughs containing the word "Toronto"**

In [18]:
df4=df3[df3['Borough'].str.contains('Toronto')]
df5=df4.reset_index(drop=True)
df5.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


**Verify the number of rows and columns the dataset now has**

In [19]:
df5.shape

(39, 5)

There are 39 rows and 5 columns

**Cluster the data using "Borough"**

In [20]:
df5['Borough'].value_counts()

Downtown Toronto    19
Central Toronto      9
West Toronto         6
East Toronto         5
Name: Borough, dtype: int64

**Add a new column called Label for the newly obtained information**

In [21]:
df5['Label']=df5['Borough'].replace(to_replace=['Downtown Toronto','Central Toronto','West Toronto','East Toronto'],value=[1,2,3,4],inplace=False)
df5.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Label
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,4
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,4
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572,4
3,M4M,East Toronto,Studio District,43.659526,-79.340923,4
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,2


**Time to visualize the data!!**

**Find the coordinates for Toronto**

In [22]:
address = 'Toronto'
geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(f'The geograpical coordinate of Toronto are {latitude}, {longitude}.')

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


**Prepare the information needed for the cluster map**

In [23]:
kclusters=len(df5.Label.unique())
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

In [24]:
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

In [25]:
markers_colors = []
for lat, lon, cluster in zip(df5['Latitude'], df5['Longitude'], df5['Label']):
    label = folium.Popup(str(df5['Borough']) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_toronto)

**View your map!**

Keep in mind Folium maps will not show in Github

In [26]:
map_toronto

**Use Foursquare API to Explore Neighborhoods**

In [27]:
CLIENT_ID = '5IIK231X0DEEK3TNKC4M5WTRGMHNHVB2OQTJFS0PSEDF2KHV' # your Foursquare ID
CLIENT_SECRET = 'CE0LOBIYOIPSXEJBA0CDLYDJ2J1D4SJ3B2UY0J451J45CUW0' # your Foursquare Secret
ACCESS_TOKEN = 'DCSEQX4E3H4R3TTJ1UHDY2XGK1HFT2RUZQLGRHVHIUQK24SM' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 5IIK231X0DEEK3TNKC4M5WTRGMHNHVB2OQTJFS0PSEDF2KHV
CLIENT_SECRET:CE0LOBIYOIPSXEJBA0CDLYDJ2J1D4SJ3B2UY0J451J45CUW0


**Search for the top 100 venues within a 2000 meter radius**

In [34]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(df5['Latitude'], df5['Longitude'], df5['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [35]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(3873, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,The Beaches,43.676357,-79.293031,Tori's Bakeshop,43.672114,-79.290331,Vegetarian / Vegan Restaurant
1,The Beaches,43.676357,-79.293031,The Fox Theatre,43.672801,-79.287272,Indie Movie Theater
2,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
3,The Beaches,43.676357,-79.293031,The Beech Tree,43.680493,-79.288846,Gastropub
4,The Beaches,43.676357,-79.293031,Beaches Bake Shop,43.680363,-79.289692,Bakery


**View Venues by Neighboorhood**

In [36]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Berczy Park,100,100,100,100,100,100
"Brockton, Parkdale Village, Exhibition Place",100,100,100,100,100,100
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",100,100,100,100,100,100
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",100,100,100,100,100,100
Central Bay Street,100,100,100,100,100,100
Christie,100,100,100,100,100,100
Church and Wellesley,100,100,100,100,100,100
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,100,100,100,100,100,100


**Check how many unique categories there are**

In [37]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 229 uniques categories.


In [39]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:50]

array(['Vegetarian / Vegan Restaurant', 'Indie Movie Theater', 'Trail',
       'Gastropub', 'Bakery', 'Ice Cream Shop', 'Bagel Shop',
       'Toy / Game Store', 'Park', 'Coffee Shop', 'Pub',
       'Health Food Store', 'French Restaurant', 'Beach',
       'Japanese Restaurant', 'Breakfast Spot', 'Nail Salon',
       'Middle Eastern Restaurant', 'Mexican Restaurant', 'BBQ Joint',
       'Skating Rink', 'Liquor Store', 'Juice Bar',
       'Hungarian Restaurant', 'Cupcake Shop', 'Mediterranean Restaurant',
       'Café', 'Diner', 'Bar', 'Caribbean Restaurant', 'Tea Room',
       'Dog Run', 'Gym', 'Playground', 'Golf Course', 'Burger Joint',
       'Indian Restaurant', 'Thai Restaurant', 'Sandwich Place',
       'Fish & Chips Shop', 'Grocery Store', 'Ramen Restaurant',
       'Chocolate Shop', 'Pharmacy', 'Bookstore', 'Bank',
       'Sushi Restaurant', 'Greek Restaurant', 'Pizza Place',
       'Asian Restaurant'], dtype=object)

In [40]:
# check if the results contain "Restaurant"
"Neighborhood" in venues_df['VenueCategory'].unique()

True

**Do an Analysis of the Neighborhoods**

In [41]:
# one hot encoding
kl_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
kl_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [kl_onehot.columns[-1]] + list(kl_onehot.columns[:-1])
kl_onehot = kl_onehot[fixed_columns]

print(kl_onehot.shape)
kl_onehot.head()

(3873, 230)


,Neighborhoods,Airport,Airport Lounge,American Restaurant,Amphitheater,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Bistro,Bookstore,Botanical Garden,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Rec Center,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Dessert Shop,Diner,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,Hobby Shop,Hostel,Hotel,Hotel Bar,Hungarian Restaurant,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lingerie Store,Liquor Store,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Nudist Beach,Organic Grocery,Other Great Outdoors,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pide Place,Pie Shop,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Racetrack,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Stationery Store,Steakhouse,Street Art,Supermarket,Sushi Restaurant,Syrian Restaurant,Taco Place,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tattoo Parlor,Tea Room,Thai Restaurant,Theater,Theme Park,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wings Joint,Yoga Studio,Zoo
0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,

In [42]:
kl_grouped = kl_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(kl_grouped.shape)
kl_grouped

(39, 230)


,Neighborhoods,Airport,Airport Lounge,American Restaurant,Amphitheater,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Bistro,Bookstore,Botanical Garden,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Rec Center,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Dessert Shop,Diner,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,Hobby Shop,Hostel,Hotel,Hotel Bar,Hungarian Restaurant,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lingerie Store,Liquor Store,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Nudist Beach,Organic Grocery,Other Great Outdoors,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pide Place,Pie Shop,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Racetrack,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Stationery Store,Steakhouse,Street Art,Supermarket,Sushi Restaurant,Syrian Restaurant,Taco Place,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tattoo Parlor,Tea Room,Thai Restaurant,Theater,Theme Park,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wings Joint,Yoga Studio,Zoo
0,Berczy Park,0.00,0.00,0.01,0.00,0.00,0.00,0.02,0.00,0.000000,0.00,0.000000,0.0000,0.00,0.030000,0.000000,0.00,0.01,0.01,0.01,0.00,0.0000,0.000000,0.00,0.010000,0.00,0.00,0.00,0.000000,0.010,0.00,0.000000,0.000000,0.00,0.000000,0.0000,0.00,0.070000,0.00,0.0000,0.00,0.00,0.00,0.00,0.00,0.010000,0.00,0.01,0.00,0.00,0.010000,0.0100,0.100000,0.00,0.0000,0.00,0.00,0.00,0.0000,0.010000,0.00,0.01,0.00,0.0000,0.00,0.0100,0.01,0.00,0.010000,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.000,0.0000,0.00,0.0300,0.010000,0.00,0.00,0.00,0.00,0.000000,0.00,0.000000,0.01,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.010000,0.00,0.000000,0.00,0.00,0.000000,0.00,0.00,0.010000,0.01,0.000000,0.00,0.00,0.00,0.00,0.01,0.000000,0.00,0.03,0.00,0.00,0.0

In [43]:
len(kl_grouped[kl_grouped["Restaurant"] > 0])

36

**Create a Seperate Dataframe for Restaurants**

In [44]:
kl_rest = kl_grouped[["Neighborhoods","Restaurant"]]

In [45]:
kl_rest.head()

,Neighborhoods,Restaurant
0,Berczy Park,0.04
1,"Brockton, Parkdale Village, Exhibition Place",0.05
2,"Business reply mail Processing Centre, South C...",0.02
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.00
4,Central Bay Street,0.06


**Cluster the Neighborhoods**

In [46]:
# set number of clusters
kclusters = 3

kl_clustering = kl_rest.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(kl_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 2, 1, 0, 2, 1, 1, 2, 2, 2], dtype=int32)

In [47]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
kl_merged = kl_rest.copy()

# add clustering labels
kl_merged["Cluster Labels"] = kmeans.labels_

In [48]:
kl_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
kl_merged.head()

,Neighborhood,Restaurant,Cluster Labels
0,Berczy Park,0.04,2
1,"Brockton, Parkdale Village, Exhibition Place",0.05,2
2,"Business reply mail Processing Centre, South C...",0.02,1
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.00,0
4,Central Bay Street,0.06,2


In [50]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
kl_merged = kl_merged.join(df5.set_index("Neighborhood"), on="Neighborhood")

print(kl_merged.shape)
kl_merged.head() # check the last columns!

(39, 8)


,Neighborhood,Restaurant,Cluster Labels,PostalCode,Borough,Latitude,Longitude,Label
0,Berczy Park,0.04,2,M5E,Downtown Toronto,43.644771,-79.373306,1
1,"Brockton, Parkdale Village, Exhibition Place",0.05,2,M6K,West Toronto,43.636847,-79.428191,3
2,"Business reply mail Processing Centre, South C...",0.02,1,M7Y,East Toronto,43.662744,-79.321558,4
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.00,0,M5V,Downtown Toronto,43.628947,-79.394420,1
4,Central Bay Street,0.06,2,M5G,Downtown Toronto,43.657952,-79.387383,1


In [51]:
# sort the results by Cluster Labels
print(kl_merged.shape)
kl_merged.sort_values(["Cluster Labels"], inplace=True)
kl_merged

(39, 8)


,Neighborhood,Restaurant,Cluster Labels,PostalCode,Borough,Latitude,Longitude,Label
28,"Runnymede, Swansea",0.010000,0,M6S,West Toronto,43.651571,-79.484450,3
10,"Dufferin, Dovercourt Village",0.000000,0,M6H,West Toronto,43.669005,-79.442259,3
33,"Summerhill West, Rathnelly, South Hill, Forest...",0.010000,0,M4V,Central Toronto,43.686412,-79.400049,2
16,"India Bazaar, The Beaches West",0.010000,0,M4L,East Toronto,43.668999,-79.315572,4
35,The Beaches,0.000000,0,M4E,East Toronto,43.676357,-79.293031,4
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0,M5V,Downtown Toronto,43.628947,-79.394420,1
34,"The Annex, North Midtown, Yorkville",0.020000,1,M5R,Central Toronto,43.672710,-79.405678,2
32,Studio District,0.020000,1,M4M,East Toronto,43.659526,-79.340923,4
30,"St. James Town, Cabbagetown",0.030000,1,M4X,Downtown Toronto,43.667967,-79.367675,1
17,"Kensington Market, Chinatown, Grange Park",0.030000,1,M5T,Downtown Toronto,43.653206,-79.400049,1


**Visualize the Results**

In [52]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(kl_merged['Latitude'], kl_merged['Longitude'], kl_merged['Neighborhood'], kl_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [53]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

**Analyze the Clusters**

In [54]:
kl_merged.loc[kl_merged['Cluster Labels'] == 0]

,Neighborhood,Restaurant,Cluster Labels,PostalCode,Borough,Latitude,Longitude,Label
28,"Runnymede, Swansea",0.01,0,M6S,West Toronto,43.651571,-79.484450,3
10,"Dufferin, Dovercourt Village",0.00,0,M6H,West Toronto,43.669005,-79.442259,3
33,"Summerhill West, Rathnelly, South Hill, Forest...",0.01,0,M4V,Central Toronto,43.686412,-79.400049,2
16,"India Bazaar, The Beaches West",0.01,0,M4L,East Toronto,43.668999,-79.315572,4
35,The Beaches,0.00,0,M4E,East Toronto,43.676357,-79.293031,4
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.00,0,M5V,Downtown Toronto,43.628947,-79.394420,1


In [55]:
kl_merged.loc[kl_merged['Cluster Labels'] == 1]

,Neighborhood,Restaurant,Cluster Labels,PostalCode,Borough,Latitude,Longitude,Label
34,"The Annex, North Midtown, Yorkville",0.020000,1,M5R,Central Toronto,43.672710,-79.405678,2
32,Studio District,0.020000,1,M4M,East Toronto,43.659526,-79.340923,4
30,"St. James Town, Cabbagetown",0.030000,1,M4X,Downtown Toronto,43.667967,-79.367675,1
17,"Kensington Market, Chinatown, Grange Park",0.030000,1,M5T,Downtown Toronto,43.653206,-79.400049,1
26,Rosedale,0.020000,1,M4W,Downtown Toronto,43.679563,-79.377529,1
36,"The Danforth West, Riverdale",0.020000,1,M4K,East Toronto,43.679557,-79.352188,4
24,"Regent Park, Harbourfront",0.030000,1,M5A,Downtown Toronto,43.654260,-79.360636,1
23,"Queen's Park, Ontario Provincial Government",0.030000,1,M7A,Downtown Toronto,43.662301,-79.389494,1
21,"North Toronto West, Lawrence Park",0.020000,1,M4R,Central Toronto,43.715383,-79.405678,2
18,Lawrence Park,0.021505,1,M4N,Central Toronto,43.728020,-79.388790,2


In [56]:
kl_merged.loc[kl_merged['Cluster Labels'] == 2]

,Neighborhood,Restaurant,Cluster Labels,PostalCode,Borough,Latitude,Longitude,Label
14,"Harbourfront East, Union Station, Toronto Islands",0.04,2,M5J,Downtown Toronto,43.640816,-79.381752,1
1,"Brockton, Parkdale Village, Exhibition Place",0.05,2,M6K,West Toronto,43.636847,-79.428191,3
4,Central Bay Street,0.06,2,M5G,Downtown Toronto,43.657952,-79.387383,1
31,Stn A PO Boxes,0.04,2,M5W,Downtown Toronto,43.646435,-79.374846,1
7,"Commerce Court, Victoria Hotel",0.05,2,M5L,Downtown Toronto,43.648198,-79.379817,1
29,St. James Town,0.04,2,M5C,Downtown Toronto,43.651494,-79.375418,1
8,Davisville,0.04,2,M4S,Central Toronto,43.704324,-79.388790,2
9,Davisville North,0.04,2,M4P,Central Toronto,43.712751,-79.390197,2
25,"Richmond, Adelaide, King",0.05,2,M5H,Downtown Toronto,43.650571,-79.384568,1
11,"First Canadian Place, Underground city",0.05,2,M5X,Downtown Toronto,43.648429,-79.382280,1
